### Se realizan diferentes transformaciones sobre las distintas columnas con el fin de limpiar los datos y disponibilzarlos para su analisis. Se trabajan sobre los dataframes de Google y Yelp con el fin de unir ambos una vez finalizado el proceso de limpieza.


## Business Google

Debido a la alta demanda de almacenamieto se va a trabajar un solo json de los 11 a manera de ejemplo para mostar las funciones.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_json('1.json', lines=True)
df2 = pd.read_json('2.json', lines=True)
df3 = pd.read_json('3.json', lines=True)
df4 = pd.read_json('4.json', lines=True)
df5 = pd.read_json('5.json', lines=True)
df6 = pd.read_json('6.json', lines=True)
df7 = pd.read_json('7.json', lines=True)
df8 = pd.read_json('8.json', lines=True)
df9 = pd.read_json('9.json', lines=True)
df10 = pd.read_json('10.json', lines=True)
df11 = pd.read_json('11.json', lines=True)

In [ ]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], ignore_index=True)

In [ ]:
# Se eliminan columnas irrelevantes.

df = df.drop(columns=['url', 'state', 'relative_results', 'latitude', 'longitude', 'description', 'price', 'hours']) 


In [ ]:
# Se convierte cada lista de las filas de la columna 'category' en una cadena unida por comas.

df['category'] = df['category'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '') 

In [ ]:
# Se crea la columna 'city' y se extrae la ciudad de la columna 'addres' diviendo esta mediante la coma y se toma el penultimo elemento.

df['city'] = df['address'].apply(lambda x: x.split(', ')[-2] if isinstance(x, str) and ', ' in x else None) 

In [ ]:
# Se define una expresion regular para buscar patrones en la columna 'address'. Luego se crea la columna 'state' donde se van a alojar los patrones encontrados.

estado_regex = r',\s*([A-Za-z]{2,})\s*(\d{5})?$' 

df['state'] = df['address'].str.extract(estado_regex, expand=True)[0] 

In [ ]:
# Se crea la columna 'zipcode' y meidiante una expresion regular a la columna 'address' se extrae el codigo postal.

df['zipcode'] = df['address'].str.extract(r'(\d+)$') 

In [ ]:
# Se aplica una expresion regular a la columna address para traer la direccion del comercio.

df['address'] = df['address'].str.extract(r',\s*(.*?)\s*,') 

In [ ]:
#Se realiza un mapeo en los estados a utilizar, cambiandoles sus siglas por sus nombres.

mapeo_estados = {'TX': 'Texas', 'CA': 'California', 'IA': 'Iowa', 'IN': 'Indiana', 'KY':'Kentucky', 'MS':'Mississippi'} 

df['state'] = df['state'].replace(mapeo_estados) 

In [ ]:
# Finalmente filtramos por los estados a usar.

estados_requeridos = ["Texas", "California", "Indiana", "Kentucky", "Iowa", "Mississippi"] 

df = df.loc[df['state'].isin(estados_requeridos), :] 

In [ ]:
# Como vamos a trabajar por restaurantes, filtramos las categorias donde aparezcan la palabra 'Restaurant'.

palabras_clave = ["Restaurants", "restaurants", "Restaurant", "restaurant"] 

condiciones = df['category'].str.contains('|'.join(palabras_clave), case=False) 

df7 = df[condiciones] 

In [ ]:
# Se establece una funcion que nos recorra el diccionario de cada fila de la columna 'MISC', con el fin de traernos los values de la key 'Service Options' en una nueva columna llamada 'Service Options'

def get_service_options(dic): 

    if isinstance(dic, dict) and "Service options" in dic: 

        return dic["Service options"] 

    else: 

        return [] 

df["Service options"] = df["MISC"].apply(lambda x: get_service_options(x)) 

In [ ]:
# Se crea la columna Delivery y se la completa con 1 o 0, dependiendo de la palabra 'Delivery' se encuentra en la columna 'Service Options'.

df['Delivery'] = df['Service options'].apply(lambda x: 1 if 'Delivery' in x else 0) 

In [ ]:
# Finalmente se eliminan las columnas que no vamos a utilizar mas.

df7.drop(columns=['MISC', 'Service options'], inplace=True) 

In [ ]:
# Se añade la columna APP y se le ingresa el valor de 'Google' para identificar este df.

df['APP'] = 'Google'

In [ ]:
# Se renombran las columnas.

df.rename(columns={'gmap_id':'ID_Business', 'avg_rating':'Stars', 'num_of_reviews':'Review_count', 'name':'Name', 'address':'Address', 'category':'Category', 'city':'City', 'state':'State', 'zipcode':'Postal_code'}, inplace=True)

In [ ]:
# Se ordenan las columnas.

df = df[['ID_Business', 'Name', 'Address', 'City', 'State', 'Postal_code', 'Category', 'Stars', 'Review_count', 'Delivery', 'APP']]

## Yelp Business

In [ ]:
business = pd.read_pickle('business.pkl')

In [ ]:
#Filtramos el dataframe con las columnas únicas ya que el dataframe business tiene columnas duplicadas
business_filtro = business.iloc[:, :-14]

In [ ]:
#Eliminamos las columnas que no son relevante para nuestro análisis.
business_filtro = business_filtro.drop(columns=['address'])
business_filtro = business_filtro.drop(columns=['postal_code'])
business_filtro = business_filtro.drop(columns=['hours'])
business_filtro = business_filtro.drop(columns=['latitude'])
business_filtro = business_filtro.drop(columns=['longitude'])

In [ ]:
# Verificamos que la columna 'categories' no contenga valores nulos
business_filtro = business_filtro.dropna(subset=['categories'])

# Filtrar las filas que contienen la palabra 'Restaurants' en la columna 'categories'
restaurantes_filtrados = business_filtro[business_filtro['categories'].str.contains('Restaurants')]

In [ ]:
# Hacer una copia del DataFrame para evitar la advertencia "SettingWithCopyWarning"
restaurantes_copia = restaurantes_filtrados.copy()

# Aplicar la función str() a cada valor del diccionario en la columna 'attributes' de la copia
restaurantes_copia['attributes'] = restaurantes_copia['attributes'].apply(str)

# Ahora, la columna 'attributes' de la copia contiene cadenas de texto en lugar de diccionarios

# Convertir la columna 'attributes' en una lista de cadenas
restaurantes_copia['attributes'] = restaurantes_copia['attributes'].tolist()
# Eliminar las llaves '{' y '}' y las comillas simples de cada cadena en la columna 'attributes'
restaurantes_copia['attributes'] = restaurantes_copia['attributes'].str.replace('[{}\'\']', '', regex=True)
# Convertir la columna 'attributes' en una lista de cadenas
restaurantes_copia['attributes'] = restaurantes_copia['attributes'].tolist()

In [ ]:
# Definir una función para procesar cada fila de la columna 'attributes' y determinar el valor de 'Delivery'
def determine_delivery_value(row):
    if pd.notnull(row):
        if 'RestaurantsDelivery: False' in row:
            return 0
        elif 'RestaurantsDelivery: True' in row:
            return 1
    return False

# Aplicar la función para obtener la nueva columna 'Delivery'
restaurantes_copia['Delivery'] = restaurantes_copia['attributes'].apply(determine_delivery_value)

# Rellenar los valores nulos en la columna 'Delivery' con el valor booleano 'False'
restaurantes_copia['Delivery'] = restaurantes_copia['Delivery'].fillna(0)

# Ahora, 'restaurantes_copia' contiene la nueva columna 'Delivery' con los valores booleanos según las condiciones especificadas

In [ ]:
# Se elimina la columna 'attributes'

restaurantes_copia.drop(columns=['is_open','attributes'], inplace=True)

In [ ]:
mapeo_estados = {'TX': 'Texas', 'CA': 'California', 'IA': 'Iowa', 'IN': 'Indiana', 'KY':'Kentucky', 'MS':'Mississippi'}

restaurantes_copia['state'] = restaurantes_copia['state'].replace(mapeo_estados)

In [ ]:
estados_requeridos = ["Texas", "California", "Indiana", "Kentucky", "Iowa", "Mississippi"]
restaurantes_copia = restaurantes_copia.loc[restaurantes_copia['state'].isin(estados_requeridos), :]

In [ ]:
# Se renombran las columnas.

restaurantes_copia.rename(columns={'business_id	':'ID_Business', 'stars':'Stars', 'review_count	':'Review_count', 'name':'Name', 'address':'Address', 'categories':'Category', 'city':'City', 'state':'State', 'postal_code':'Postal_code'}, inplace=True)

In [ ]:
restaurantes_copia['APP'] = 'Yelp'

In [ ]:
# Se ordenan las columnas.

restaurantes_copia = restaurantes_copia[['ID_Business', 'Name', 'Address', 'City', 'State', 'Postal_code', 'Category', 'Stars', 'Review_count', 'Delivery', 'APP']]

In [ ]:
# Concatenamos los df.
business = pd.concat([df, restaurantes_copia], ignore_index=True)

In [ ]:
# Reseteamos el indice.
business = business.reset_index(drop=True)

Guardamos los resultados en parquet.

In [ ]:
df_google_reviews.to_parquet('business_google_yelp.parquet')